In [1]:
# trades_cifar10_fixed.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from torchvision.models import resnet18
import numpy as np
import random
from tqdm import tqdm

# --------------------------
# 1. Reproducibility
# --------------------------
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --------------------------
# 2. Hyperparameters
# --------------------------
batch_size = 128
learning_rate = 0.01  # smaller for stability
num_epochs = 80
epsilon = 8/255  # L_inf perturbation
alpha = 2/255    # PGD step size
num_steps = 10   # PGD steps
beta = 6.0       # TRADES trade-off

# --------------------------
# 3. CIFAR-10 Data
# --------------------------
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# --------------------------
# 4. Model
# --------------------------
model = resnet18(num_classes=10).to(device)

# --------------------------
# 5. TRADES Loss Function
# --------------------------
def trades_loss(model, x_natural, y):
    model.train()
    batch_size = len(x_natural)

    # Initialize adversarial example
    x_adv = x_natural.detach() + 0.001 * torch.randn_like(x_natural)
    for _ in range(num_steps):
        x_adv.requires_grad_()
        logits_adv = model(x_adv)
        logits_nat = model(x_natural)
        loss_kl = F.kl_div(F.log_softmax(logits_adv, dim=1),
                           F.softmax(logits_nat, dim=1),
                           reduction='batchmean')
        grad = torch.autograd.grad(loss_kl, [x_adv])[0]
        x_adv = x_adv + alpha * torch.sign(grad)
        x_adv = torch.min(torch.max(x_adv, x_natural - epsilon), x_natural + epsilon)
        x_adv = torch.clamp(x_adv, 0.0, 1.0).detach()

    # Compute TRADES loss
    logits = model(x_natural)
    logits_adv = model(x_adv)
    loss_ce = F.cross_entropy(logits, y)
    loss_kl = F.kl_div(F.log_softmax(logits_adv, dim=1),
                       F.softmax(logits, dim=1),
                       reduction='batchmean')
    loss = loss_ce + beta * loss_kl
    return loss

# --------------------------
# 6. Optimizer & Scheduler
# --------------------------
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 25], gamma=0.1)

# --------------------------
# 7. Evaluation Function
# --------------------------
def evaluate(model, loader, adversarial=False):
    model.eval()
    correct = 0
    total = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        if adversarial:
            x_adv = images.detach() + 0.001 * torch.randn_like(images)
            for _ in range(num_steps):
                x_adv.requires_grad_()
                logits = model(x_adv)
                loss = F.cross_entropy(logits, labels)
                grad = torch.autograd.grad(loss, [x_adv])[0]
                x_adv = x_adv + alpha * torch.sign(grad)
                x_adv = torch.min(torch.max(x_adv, images - epsilon), images + epsilon)
                x_adv = torch.clamp(x_adv, 0.0, 1.0).detach()
            outputs = model(x_adv)
        else:
            outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    acc = 100.0 * correct / total
    return acc

# --------------------------
# 8. Training Loop
# --------------------------
print("Starting TRADES training...")
for epoch in range(1, num_epochs+1):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = trades_loss(model, images, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    scheduler.step()
    avg_loss = running_loss / len(train_loader.dataset)
    clean_acc = evaluate(model, test_loader, adversarial=False)
    adv_acc = evaluate(model, test_loader, adversarial=True)
    print(f"Epoch [{epoch}/{num_epochs}] | Loss: {avg_loss:.4f} | Clean Acc: {clean_acc:.2f}% | Adv Acc: {adv_acc:.2f}%")

# --------------------------
# 9. Save Model
# --------------------------
torch.save(model.state_dict(), "trades_resnet18_cifar10_fixed.pth")
print("Training completed and model saved!")


Using device: cuda
Starting TRADES training...


Epoch 1/80: 100%|██████████| 391/391 [02:20<00:00,  2.77it/s]


Epoch [1/80] | Loss: 2.2856 | Clean Acc: 34.00% | Adv Acc: 19.72%


Epoch 2/80: 100%|██████████| 391/391 [02:19<00:00,  2.80it/s]


Epoch [2/80] | Loss: 2.0289 | Clean Acc: 36.84% | Adv Acc: 18.69%


...

Epoch [55/80] | Loss: 1.6895 | Clean Acc: 52.84% | Adv Acc: 28.01%


Epoch 56/80: 100%|██████████| 391/391 [02:19<00:00,  2.80it/s]


Epoch [56/80] | Loss: 1.6887 | Clean Acc: 54.35% | Adv Acc: 28.63%


Epoch 57/80:  53%|█████▎    | 207/391 [01:14<01:05,  2.79it/s]


KeyboardInterrupt: 

In [2]:
torch.save(model.state_dict(), "trades_resnet18_cifar10_fixed.pth")
print("Training completed and model saved!")

Training completed and model saved!
